In [1]:
from syft import DataSubjectList

/home/e/anaconda3/envs/Hagrid/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/e/PycharmProjects/PySyft/packages/syft/src/syft/core/common/serde/serializable.py:105: UserWarning: __attr_allowlist__ not defined for type Activation, even if it uses recursive serde, defaulting on the empty list.
  warnings.warn(
/home/e/PycharmProjects/PySyft/packages/syft/src/syft/core/common/serde/serializable.py:105: UserWarning: __attr_allowlist__ not defined for type Linear, even if it uses recursive serde, defaulting on the empty list.
  warnings.warn(
/home/e/PycharmProjects/PySyft/packages/syft/src/syft/core/common/serde/serializable.py:105: UserWarning: __attr_allowlist__ not defined for type BatchNorm, even if it uses recursive serde, defaulting on the empty list.
  warnings.warn(
/home/e/PycharmProjects/PySy

In [2]:
import numpy as np

In [3]:
dsl = DataSubjectList(one_hot_lookup=np.arange(10, 20), data_subjects_indexed=np.random.choice(np.arange(10), (10,10)))

In [4]:
dsl.one_hot_lookup

array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])

In [5]:
dsl.data_subjects_indexed

array([[4, 1, 7, 0, 6, 5, 4, 8, 0, 8],
       [4, 8, 1, 2, 8, 8, 3, 2, 5, 4],
       [5, 9, 3, 2, 9, 9, 9, 5, 8, 3],
       [2, 8, 9, 1, 8, 3, 8, 9, 7, 2],
       [0, 9, 8, 6, 7, 1, 0, 7, 3, 3],
       [2, 0, 5, 0, 7, 4, 7, 0, 3, 6],
       [1, 1, 9, 3, 4, 7, 0, 2, 5, 5],
       [8, 1, 9, 5, 8, 8, 5, 4, 7, 4],
       [2, 4, 6, 1, 7, 3, 0, 8, 9, 4],
       [1, 9, 2, 2, 1, 0, 2, 7, 0, 4]])

In [6]:
dsl[0].one_hot_lookup

array([10, 11, 14, 15, 16, 17, 18])

In [7]:
dsl[0].data_subjects_indexed

array([4, 1, 7, 0, 6, 5, 4, 8, 0, 8])

In [8]:
np.unique(dsl[0].one_hot_lookup)

array([10, 11, 14, 15, 16, 17, 18])

In [9]:
np.unique(dsl.data_subjects_indexed[0])

array([0, 1, 4, 5, 6, 7, 8])

In [10]:
dsl[0, 0].one_hot_lookup

array([14])

In [11]:
dsl[0, 0].data_subjects_indexed

4

In [12]:
dsl[0, 0] = DataSubjectList(one_hot_lookup=np.array([10]), data_subjects_indexed=np.array([0]))

In [13]:
dsl.one_hot_lookup

array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])

In [14]:
dsl.data_subjects_indexed

array([[0, 1, 7, 0, 6, 5, 4, 8, 0, 8],
       [4, 8, 1, 2, 8, 8, 3, 2, 5, 4],
       [5, 9, 3, 2, 9, 9, 9, 5, 8, 3],
       [2, 8, 9, 1, 8, 3, 8, 9, 7, 2],
       [0, 9, 8, 6, 7, 1, 0, 7, 3, 3],
       [2, 0, 5, 0, 7, 4, 7, 0, 3, 6],
       [1, 1, 9, 3, 4, 7, 0, 2, 5, 5],
       [8, 1, 9, 5, 8, 8, 5, 4, 7, 4],
       [2, 4, 6, 1, 7, 3, 0, 8, 9, 4],
       [1, 9, 2, 2, 1, 0, 2, 7, 0, 4]])

In [15]:
dsl[0] = DataSubjectList(one_hot_lookup=np.array([10]), data_subjects_indexed=np.zeros_like(dsl.data_subjects_indexed[0]))

In [16]:
dsl.data_subjects_indexed

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [4, 8, 1, 2, 8, 8, 3, 2, 5, 4],
       [5, 9, 3, 2, 9, 9, 9, 5, 8, 3],
       [2, 8, 9, 1, 8, 3, 8, 9, 7, 2],
       [0, 9, 8, 6, 7, 1, 0, 7, 3, 3],
       [2, 0, 5, 0, 7, 4, 7, 0, 3, 6],
       [1, 1, 9, 3, 4, 7, 0, 2, 5, 5],
       [8, 1, 9, 5, 8, 8, 5, 4, 7, 4],
       [2, 4, 6, 1, 7, 3, 0, 8, 9, 4],
       [1, 9, 2, 2, 1, 0, 2, 7, 0, 4]])

In [17]:
a = np.array([1,2,3,4])
b = np.array([5,6,7,4])
np.isin(a, b)

array([False, False, False,  True])

In [18]:
np.append(a,b)

array([1, 2, 3, 4, 5, 6, 7, 4])

In [19]:
overlapping_indices = np.isin(a, b)
unique_indices = (overlapping_indices == False).nonzero()[0]  # noqa: E712

In [20]:
unique_indices

array([0, 1, 2])

In [21]:
search = overlapping_indices == True
search

array([False, False, False,  True])

In [22]:
np.invert(search).nonzero()[0]

array([0, 1, 2])